# 1. Importing Required Modules

In [1]:
from zipfile import ZipFile 
import pandas as pd
import os
import tabula

# 2. Extract data file from zip files

## 2.a. Create a function for data extraction

In [2]:
def security_data_consolidator(path_folder):

    # Listing all zip files in the directory
    securities_mf_path = path_folder
    securities_mf_files = os.listdir(securities_mf_path)

    # Create an empty text file
    securities_mf_all_filename = securities_mf_path+'.txt'
    with open(os.path.join(securities_mf_path, securities_mf_all_filename), 'a', encoding="utf-8") as securities_mf_file:
        pass

    # Create temprary folder
    mf_temp_dir = 'tmp'
    if not os.path.exists(os.path.join(securities_mf_path, mf_temp_dir)):
        os.makedirs(os.path.join(securities_mf_path, mf_temp_dir))

    # Track the file processed
    file_processed = 0

    # Iterate over all files in dataset folder. 
    # Extract the txt file and put it in the tmp folder
    # Open the txt file, append its content to the empty file previously created
    # Delete the file in tmp folder
    for file in securities_mf_files:
        # Load the zip file and create a zip object 
        with ZipFile(os.path.join(securities_mf_path, file), 'r') as mf_zip: 
    
            # Extracting all the members of the zip  
            # into a specific location. 
            mf_zip.extractall(path=os.path.join(securities_mf_path, mf_temp_dir)) 
        
        # Close zip file
        mf_zip.close()

        # Open the file in tmp folder, read the content, and delete the file once its done
        securities_mf_tmp = os.listdir(os.path.join(securities_mf_path, mf_temp_dir))
        securities_mf_txt = open(os.path.join(os.path.join(securities_mf_path, mf_temp_dir),securities_mf_tmp[0]),"r", encoding="utf-8")
        file_content = securities_mf_txt.read()
        securities_mf_txt.close()
        os.remove(os.path.join(os.path.join(securities_mf_path, mf_temp_dir),securities_mf_tmp[0]))
        
        # Only the header line on the first file needs to be added to the new file
        if file_processed==0:
            all_mf_file = open(os.path.join(securities_mf_path, securities_mf_all_filename), 'a', encoding="utf-8")
            all_mf_file.write(file_content)
            all_mf_file.close()
        else:
            file_content_list = file_content.split('\n')[1:]
            all_mf_file = open(os.path.join(securities_mf_path, securities_mf_all_filename), 'a', encoding="utf-8")
            all_mf_file.write("\n".join(file_content_list))
            all_mf_file.close()
        
        # Increase file counter
        file_processed+=1

    # Delete tmp folder
    os.removedirs(os.path.join(securities_mf_path, mf_temp_dir))
    print(f'Success! File output: {os.path.join(securities_mf_path, securities_mf_all_filename)}')

## 2.b. Extract data from securities master file 

In [3]:
security_data_consolidator('Dataset_Masterfile_Efek')

Success! File output: Dataset_Masterfile_Efek\Dataset_Masterfile_Efek.txt


## 2.c. Extract data from securities ownership master file

In [4]:
security_data_consolidator('Dataset_Ownership_Efek')

Success! File output: Dataset_Ownership_Efek\Dataset_Ownership_Efek.txt


# 3. Read the combined data

## 3.1 Read security master file data

In [5]:
securities_mf = pd.read_csv('Dataset_Masterfile_Efek/Dataset_Masterfile_Efek.txt', sep='|')
securities_mf.head()

,Date,Code,Description,Type,Isin Code,Issuer,Status,Stock Exchange,Listing Date,Currency,...,Nominal Value,Num. of Sec,Originated Amt,Current Amt,Total Scripless,Local (%),Foreign (%),Total (%),Sector,Closing Price
0,31-MAY-2021,TRIM,TRIMEGAH SEKURITAS INDONESIA Tbk,EQUITY,ID1000052608,"TRIMEGAH SEKURITAS INDONESIA Tbk, PT",ACTIVE,IDX,NaN,IDR,...,50.0,7.109300e+09,NaN,NaN,7.109300e+09,21.17,78.83,100.00,SECURITIES COMPANY,130.0
1,31-MAY-2021,TRIN,PERINTIS TRINITI PROPERTI Tbk,EQUITY,ID1000153604,"PERINTIS TRINITI PROPERTI Tbk, PT",ACTIVE,IDX,15-JAN-2020,IDR,...,100.0,4.373364e+09,NaN,NaN,3.973864e+09,90.87,0.00,90.87,PROPERTY AND REAL ESTATE,157.0
2,31-MAY-2021,TRIO,TRIKOMSEL OKE Tbk,EQUITY,ID1000112501,PT TRIKOMSEL OKE TBK,ACTIVE,IDX,14-APR-2009,IDR,...,100.0,2.600749e+10,NaN,NaN,2.592890e+10,50.62,49.08,99.70,RETAIL TRADE,426.0
3,31-MAY-2021,TRIS,TRISULA INTERNATIONAL Tbk,EQUITY,ID1000123102,"TRISULA INTERNATIONAL Tbk, PT",ACTIVE,IDX,28-JUN-2012,IDR,...,100.0,3.141444e+09,NaN,NaN,3.646108e+08,7.62,3.99,11.61,"TEXTILE, GARMENT",144.0
4,31-MAY-2021,TRJA,TRANSKON JAYA Tbk,EQUITY,ID1000156805,"TRANSKON JAYA Tbk, PT",ACTIVE,IDX,27-AUG-2020,IDR,...,100.0,1.510200e+09,NaN,NaN,3.750000e+08,24.79,0.04,24.83,TRANSPORTATION,168.0


Identify the type of financial securities in the master file. 

In [6]:
securities_mf['Type'].unique()

array(['EQUITY', 'CORPORATE BOND', 'GOVERNMENT BOND', 'WARRANT', 'RIGHT',
       'TERM NOTES', 'NEGOTIABLE CERTIFICATE OF DEPOSIT',
       'COMMERCIAL PAPER', 'SBSN', 'SPN', 'SUKUK', 'EBA',
       'DANA INVESTASI REAL ESTATE', 'EQUITY CROWDFUNDING (ECF)',
       'DEBT CROWD FUNDING', 'SUKUK CROWD FUNDING', 'STRUCTURED WARRANT',
       'BANK INDONESIA RUPIAH SECURITIES (SRBI)',
       'Bank Indonesia Foreign Currency Securities (SVBI)',
       'Bank Indonesia Foreign Currency Sukuk (SUVBI)', 'PERPETUAL BONDS'],
      dtype=object)

In [7]:
bonds_type = ['CORPORATE BOND', 'GOVERNMENT BOND', 
              'SBSN', 'SPN', 'SUKUK', 
              'DEBT CROWD FUNDING', 'SUKUK CROWD FUNDING',
              'BANK INDONESIA RUPIAH SECURITIES (SRBI)',
              'Bank Indonesia Foreign Currency Securities (SVBI)',
              'Bank Indonesia Foreign Currency Sukuk (SUVBI)', 'PERPETUAL BONDS']

Filter the master file to only display bonds data

In [9]:
securities_mf_bond = securities_mf[securities_mf['Type'].isin(bonds_type)].reset_index(drop=True)
securities_mf_bond.head()

,Date,Code,Description,Type,Isin Code,Issuer,Status,Stock Exchange,Listing Date,Currency,...,Nominal Value,Num. of Sec,Originated Amt,Current Amt,Total Scripless,Local (%),Foreign (%),Total (%),Sector,Closing Price
0,31-MAY-2021,ABSM01A,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097A7,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,NaN,NaN,1.150000e+11,1.150000e+11,1.150000e+11,100.0,0.0,100.0,FINANCIAL INSTITUTION,1.0
1,31-MAY-2021,ABSM01B,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097B5,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,NaN,NaN,2.500000e+10,2.500000e+10,2.500000e+10,100.0,0.0,100.0,FINANCIAL INSTITUTION,1.0
2,31-MAY-2021,ABSM01C,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097C3,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,NaN,NaN,3.500000e+10,3.500000e+10,3.500000e+10,100.0,0.0,100.0,FINANCIAL INSTITUTION,1.0
3,31-MAY-2021,ADCP01A,OBLIGASI I ADHI COMMUTER PROPERTI TAHUN 2021 S...,CORPORATE BOND,IDA0001151A2,"ADHI COMMUTER PROPERTI, PT",ACTIVE,IDX,21-MAY-2021,IDR,...,NaN,NaN,4.910000e+11,4.910000e+11,4.910000e+11,100.0,0.0,100.0,PROPERTY AND REAL ESTATE,1.0
4,31-MAY-2021,ADCP01B,OBLIGASI I ADHI COMMUTER PROPERTI TAHUN 2021 S...,CORPORATE BOND,IDA0001151B0,"ADHI COMMUTER PROPERTI, PT",ACTIVE,IDX,21-MAY-2021,IDR,...,NaN,NaN,9.000000e+09,9.000000e+09,9.000000e+09,100.0,0.0,100.0,PROPERTY AND REAL ESTATE,1.0


In [11]:
securities_mf_bond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43405 entries, 0 to 43404
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             43405 non-null  object 
 1   Code             43405 non-null  object 
 2   Description      43405 non-null  object 
 3   Type             43405 non-null  object 
 4   Isin Code        43405 non-null  object 
 5   Issuer           43405 non-null  object 
 6   Status           43405 non-null  object 
 7   Stock Exchange   39433 non-null  object 
 8   Listing Date     39397 non-null  object 
 9   Currency         43405 non-null  object 
 10  Form             43405 non-null  object 
 11  Eff. Date Isin   50 non-null     object 
 12  Maturity Date    43405 non-null  object 
 13  Expire Date      0 non-null      object 
 14  Exercise Price   0 non-null      float64
 15  Interest         38526 non-null  float64
 16  Interest Type    43242 non-null  object 
 17  Interest Fre

## 3.2 Read Security Ownership Data

In [12]:
securities_os = pd.read_csv('Dataset_Ownership_Efek/Dataset_Ownership_Efek.txt', sep='|')
securities_os.head()

,Date,Code,Type,Sec. Num,Price,Local IS,Local CP,Local PF,Local IB,Local ID,...,Foreign IS,Foreign CP,Foreign PF,Foreign IB,Foreign ID,Foreign MF,Foreign SC,Foreign FD,Foreign OT,Total.1
0,31-MAY-2021,AALI,EQUITY,1.924688e+09,8825,114938014.0,1.127444e+07,19468622,49700,100544313,...,1403910,3855240,7316838,13323164,1423464,32725069,26900419,208822,6914300,94071226
1,31-MAY-2021,ABBA,EQUITY,2.755125e+09,246,0.0,1.692657e+09,0,0,566614547,...,0,2120900,0,14969700,1800,0,1900,0,0,17094300
2,31-MAY-2021,ABDA,EQUITY,6.208067e+08,7050,21885.0,4.753868e+07,0,98,47919272,...,0,426136555,0,97403500,400,0,0,0,62,523540517
3,31-MAY-2021,ABMM,EQUITY,2.753165e+09,825,2677000.0,6.965900e+06,1600,0,50690300,...,0,57405300,0,320433900,406600,10351700,0,0,151862900,540460400
4,31-MAY-2021,ACES,EQUITY,1.715000e+10,1495,328564522.0,9.927710e+07,20041945,0,272974411,...,21176900,715615629,1246731977,511051505,1561100,2235817184,196662025,4720799,941505231,5874842350


Filter ownership data to only display bond data

In [13]:
securities_os_bond = securities_os[securities_os['Type'].isin(bonds_type)].reset_index(drop=True)
securities_os_bond.head()

,Date,Code,Type,Sec. Num,Price,Local IS,Local CP,Local PF,Local IB,Local ID,...,Foreign IS,Foreign CP,Foreign PF,Foreign IB,Foreign ID,Foreign MF,Foreign SC,Foreign FD,Foreign OT,Total.1
0,31-MAY-2021,ABSM01A,CORPORATE BOND,1.150000e+11,1,7.500000e+08,5.000000e+08,0,0,15650000000,...,0,0,0,0,0,0,0,0,0,0
1,31-MAY-2021,ABSM01B,CORPORATE BOND,2.500000e+10,1,1.000000e+09,0.000000e+00,0,0,4000000000,...,0,0,0,0,0,0,0,0,0,0
2,31-MAY-2021,ABSM01C,CORPORATE BOND,3.500000e+10,1,0.000000e+00,0.000000e+00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31-MAY-2021,ADCP01A,CORPORATE BOND,4.910000e+11,1,0.000000e+00,1.000000e+11,10000000000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,31-MAY-2021,ADCP01B,CORPORATE BOND,9.000000e+09,1,5.000000e+09,0.000000e+00,2000000000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
securities_os_bond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40198 entries, 0 to 40197
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        40198 non-null  object 
 1   Code        40198 non-null  object 
 2   Type        40198 non-null  object 
 3   Sec. Num    38501 non-null  float64
 4   Price       40198 non-null  int64  
 5   Local IS    40198 non-null  float64
 6   Local CP    40198 non-null  float64
 7   Local PF    40198 non-null  int64  
 8   Local IB    40198 non-null  int64  
 9   Local ID    40198 non-null  int64  
 10  Local MF    40198 non-null  int64  
 11  Local SC    40198 non-null  int64  
 12  Local FD    40198 non-null  int64  
 13  Local OT    40198 non-null  int64  
 14  Total       40198 non-null  float64
 15  Foreign IS  40198 non-null  int64  
 16  Foreign CP  40198 non-null  int64  
 17  Foreign PF  40198 non-null  int64  
 18  Foreign IB  40198 non-null  int64  
 19  Foreign ID  40198 non-nul

# 4. Merge ownership and master data

Both dataframe are outer-joined using Date and Code columns.

In [18]:
bond_data = securities_mf_bond.merge(securities_os_bond, on=['Date','Code'], how='outer')
bond_data.head()

,Date,Code,Description,Type_x,Isin Code,Issuer,Status,Stock Exchange,Listing Date,Currency,...,Foreign IS,Foreign CP,Foreign PF,Foreign IB,Foreign ID,Foreign MF,Foreign SC,Foreign FD,Foreign OT,Total.1
0,31-MAY-2021,ABSM01A,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097A7,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31-MAY-2021,ABSM01B,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097B5,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31-MAY-2021,ABSM01C,OBLIGASI AB SINAR MAS MULTIFINANCE I TAHUN 202...,CORPORATE BOND,IDA0001097C3,"AB SINAR MAS MULTIFINANCE, PT",ACTIVE,IDX,07-SEP-2020,IDR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31-MAY-2021,ADCP01A,OBLIGASI I ADHI COMMUTER PROPERTI TAHUN 2021 S...,CORPORATE BOND,IDA0001151A2,"ADHI COMMUTER PROPERTI, PT",ACTIVE,IDX,21-MAY-2021,IDR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31-MAY-2021,ADCP01B,OBLIGASI I ADHI COMMUTER PROPERTI TAHUN 2021 S...,CORPORATE BOND,IDA0001151B0,"ADHI COMMUTER PROPERTI, PT",ACTIVE,IDX,21-MAY-2021,IDR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
bond_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44246 entries, 0 to 44245
Data columns (total 51 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             44246 non-null  object 
 1   Code             44246 non-null  object 
 2   Description      43405 non-null  object 
 3   Type_x           43405 non-null  object 
 4   Isin Code        43405 non-null  object 
 5   Issuer           43405 non-null  object 
 6   Status           43405 non-null  object 
 7   Stock Exchange   39433 non-null  object 
 8   Listing Date     39397 non-null  object 
 9   Currency         43405 non-null  object 
 10  Form             43405 non-null  object 
 11  Eff. Date Isin   50 non-null     object 
 12  Maturity Date    43405 non-null  object 
 13  Expire Date      0 non-null      object 
 14  Exercise Price   0 non-null      float64
 15  Interest         38526 non-null  float64
 16  Interest Type    43242 non-null  object 
 17  Interest Fre

# 5. Filter SSG Bonds

In [30]:
green_sukuk = ['ST006','ST007','ST008','ST009','ST010T4','ST011T2','ST011T4','ST012T4']

In [50]:
ssg_bond_data = bond_data[(bond_data['Description'].str.contains('BKLJ') |   # First keyword: sustainable or berkelanjutan
                          bond_data['Description'].str.contains('GREEN ') |    # Second keyword: green (with whitespace after)
                          bond_data['Description'].str.contains('LINGKUNGAN') |   # Third keyword: environment or lingkungan
                          bond_data['Code'].isin(green_sukuk)) &   # sovereign sukuk is not listed in master data, therefore it has to be identified manually
                          bond_data['Foreign OT'].notnull()   # Ownership data is not null, representing that the bond has been traded in public
                          ].reset_index(drop=True)
ssg_bond_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5603 entries, 0 to 5602
Data columns (total 51 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             5603 non-null   object 
 1   Code             5603 non-null   object 
 2   Description      5559 non-null   object 
 3   Type_x           5559 non-null   object 
 4   Isin Code        5559 non-null   object 
 5   Issuer           5559 non-null   object 
 6   Status           5559 non-null   object 
 7   Stock Exchange   5498 non-null   object 
 8   Listing Date     5498 non-null   object 
 9   Currency         5559 non-null   object 
 10  Form             5559 non-null   object 
 11  Eff. Date Isin   0 non-null      object 
 12  Maturity Date    5559 non-null   object 
 13  Expire Date      0 non-null      object 
 14  Exercise Price   0 non-null      float64
 15  Interest         5558 non-null   float64
 16  Interest Type    5559 non-null   object 
 17  Interest Freq 

# 6. Export SSG Bonds data

In [51]:
ssg_bond_data.to_csv('Indonesia_SSG_Bonds_Data.txt', sep='#',index=False)

# Extract data from PDF green sukuk performance report